In [43]:
import os
import glob
import pandas as pd
import numpy as np
import pytest

from transformers4rec import tf as tr
import tensorflow as tf
from transformers4rec.tf.ranking_metric import NDCGAt, RecallAt

In [44]:
INPUT_DATA_DIR = os.environ.get("INPUT_DATA_DIR", '../data/')

In [45]:
from merlin_standard_lib import Schema
# define schema object to pass it to the TabularSeqeunceFeatures class
SCHEMA_PATH = os.path.join(INPUT_DATA_DIR, 'schema_test.pb')
schema = Schema().from_proto_text(SCHEMA_PATH)

In [46]:
def tf_yoochoose_like():
    return tr.data.tabular_sequence_testing_data.tf_synthetic_data(
    num_rows=100, min_session_length=5, max_session_length=20
    )

In [47]:
data = tf_yoochoose_like()

In [48]:
data

{'session_id': <tf.Tensor: shape=(100,), dtype=int32, numpy=
 array([  739430,  1922385, 10977637, 10196464,  9061926,  2421346,
         6984607,  8835435,  2487530,  6919522,  4695425,  5466939,
         9000814,  4895952,  6939952,  9230367,  2773905,  8875714,
         1074610,  2750273,   275702,  8640983,  8061078,  3581699,
          816393,  1118865,  6376258,  3642239,  8819134,  6736303,
         5288552,  5925638,  9498058,  4633160,  8452541,  9793777,
        10210373,  5710682,  8826922,  9023261,  6801031,  1217510,
         8419059,  9534488, 10517804,  3523740,  2045522,  7488114,
         5838645,  7048312,  3851580,  3775423,   803113,  4742097,
         4401128,  8173258,  8147722,  6284797, 11275978,   154292,
         2320915,  1444378, 10340957,  3949215,  4975108,  4453050,
         8932625,  5569608,   445270,  5868053,  8231859,  5917790,
         1500055,  7182694,  9165339,    87824,  3919658,  5721006,
         8733123,  8326502, 10068906,  4688880,  709616

In [49]:
input_module = tr.TabularSequenceFeatures.from_schema(
    schema,
    max_sequence_length=20,
    continuous_projection=64,
    d_output=64,
    masking='causal'
)
body = tr.SequentialBlock([input_module, tr.MLPBlock([64])])
task = tr.NextItemPredictionTask(weight_tying=True, metrics=[NDCGAt(top_ks=[10, 20], labels_onehot=True)])

head = task.to_head(body, input_module)

# copy_head = test_utils.assert_serialization(head)
targets = data["item_id/list"]

model = task.to_model(body, input_module)
model.compile(optimizer="adam")
outputs = model(data)
# dataset = tf.data.Dataset.from_tensor_slices(
#         (data, targets)
#     ).batch(50)
# losses = model.fit(dataset, epochs=5)
# metrics = model.evaluate(data,targets,return_dict=True)

In [50]:
assert outputs.shape[-1] == 51997

In [51]:
input_module = tr.TabularSequenceFeatures.from_schema(
    schema,
    max_sequence_length=20,
    continuous_projection=64,
    d_output=100,
    masking="causal"
)

In [52]:
def tf_yoochoose_like():
    return tr.data.tabular_sequence_testing_data.tf_synthetic_data(
        num_rows=100, min_session_length=5, max_session_length=20
    )

In [53]:
data = tf_yoochoose_like()
body = tr.SequentialBlock(
    [input_module, tr.MLPBlock([100])],
)

task = tr.NextItemPredictionTask(weight_tying=True)
head = task.to_head(body)

loss = head.compute_loss(body_outputs=data, targets=None)

metrics = head.metric_results()

ValueError: NextItemPredictionTask needs a 3-dim vector as input, found:{'session_id': TensorShape([100]), 'session_start': TensorShape([100]), 'timestamp/age_days/list': TensorShape([100, 20]), 'timestamp/age_days/LogOp/Normalize/list': TensorShape([100, 20]), 'timestamp/hour/list': TensorShape([100, 20]), 'timestamp/weekday/list': TensorShape([100, 20]), 'timestamp/day/list': TensorShape([100, 20]), 'timestamp/month/list': TensorShape([100, 20]), 'timestamp/year/list': TensorShape([100, 20]), 'timestamp/hour/sin/list': TensorShape([100, 20]), 'timestamp/hour/cos/list': TensorShape([100, 20]), 'timestamp/weekday/sin/list': TensorShape([100, 20]), 'timestamp/weekday/cos/list': TensorShape([100, 20]), 'ts/list': TensorShape([100, 20]), 'ts/last': TensorShape([100]), 'ts/first': TensorShape([100]), 'item_id/list': TensorShape([100, 20]), 'session_size': TensorShape([100]), 'category/list': TensorShape([100, 20]), 'day_idx': TensorShape([100]), 'user_country': TensorShape([100]), 'user_ag

In [54]:
with pytest.raises(ValueError) as excinfo:
        body = tr.SequentialBlock([input_module, tr.MLPBlock([64])])
        task = tr.NextItemPredictionTask(weight_tying=True)
        head = task.to_head(body)
        _ = head(input_module)

        assert "NextItemPredictionTask needs a 3-dim vector as input, found:" in str(excinfo.value)

TypeError: object of type 'NoneType' has no len()